In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

In [2]:
df=pd.read_csv('../data/data2020.csv')
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [3]:
df1 = df.copy()
le=LabelEncoder()
categorical = df.select_dtypes(include = 'O')
categorical.columns
for feature in categorical:
    df1[feature]=le.fit_transform(df1[feature])
    print(feature)

HeartDisease
Smoking
AlcoholDrinking
Stroke
DiffWalking
Sex
AgeCategory
Race
Diabetic
PhysicalActivity
GenHealth
Asthma
KidneyDisease
SkinCancer


In [4]:
def split_data(X, Y):
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,stratify=Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,stratify=df1['HeartDisease'])
    return X_train, X_test, y_train, y_test
def over_sample_date(X_train, y_train, data_rate = 1):
    minority_class = y_train.value_counts().idxmin()    # 1
    majority_class = y_train.value_counts().idxmax()    # 0

    minority_data = X_train[y_train == minority_class]
    majority_data = X_train[y_train == majority_class]
    minority_labels = y_train[y_train == minority_class]
    majority_labels = y_train[y_train == majority_class]

    oversampled_minority_data = minority_data.sample(int(len(majority_data)*data_rate), replace=True)
    oversampled_minority_labels = minority_labels.sample(int(len(majority_data)*data_rate), replace=True)
    X_train_oversampled = pd.concat([majority_data, oversampled_minority_data])
    y_train_oversampled = pd.concat([majority_labels, oversampled_minority_labels])
    return X_train_oversampled, y_train_oversampled

## Change data_rate and other things to improve the data

In [66]:
def normalize_row(row, training_data): # insert a new data and normalize with training data and get the normalized output
    pd.concat([row, training_data])
    scaler = StandardScaler()
    training_data = scaler.fit_transform(training_data)
    return training_data[0]
X=df1.drop('HeartDisease',axis=1)
Y=df1['HeartDisease']
X_train, X_test, y_train, y_test = split_data(X,Y)
X_train_oversampled, Y_train_oversampled = over_sample_date(X_train, y_train, data_rate = 1)


In [69]:
df_new = X_train_oversampled.iloc[0]

df_new = normalize_row(df_new, X_train_oversampled)
print(df_new)

[-0.6396843   1.01743866 -0.24283626 -0.32216599  2.47242786 -0.49717844
  1.77326158 -1.05758476  1.26188349  0.46168833 -0.56236812 -1.5800868
  0.57892723  0.55411417  2.33595123 -0.2893677  -0.39308983]


In [70]:
scaler = StandardScaler()
X_train_oversampled = scaler.fit_transform(X_train_oversampled)
print(X_train_oversampled[0])
X_test = scaler.transform(X_test)

[-0.6396843   1.01743866 -0.24283626 -0.32216599  2.47242786 -0.49717844
  1.77326158 -1.05758476  1.26188349  0.46168833 -0.56236812 -1.5800868
  0.57892723  0.55411417  2.33595123 -0.2893677  -0.39308983]


In [53]:

df_new = scaler.fit_transform(df_new)

In [54]:
df_new[0]

array([-1.82421407,  1.05132024, -0.24985088,  3.36466358, -0.51544353,
        3.08195449,  1.87973398, -1.03607012,  1.01408138,  0.46725199,
       -0.5375019 ,  0.6144571 , -0.84986931,  0.56467944, -0.42111314,
       -0.27195243, -0.37791696])

In [55]:
X_train_oversampled[0]

array([-1.83305537,  1.01772584, -0.24441643,  3.12754643, -0.53868494,
        3.03133783,  1.77131768, -1.05855574,  0.96123949,  0.45983411,
       -0.56692066,  0.63357223, -0.86652053,  0.54926309, -0.42799676,
       -0.28910303, -0.39277088])

In [32]:
log_reg = LogisticRegression()
log_reg.fit(X_train_oversampled, Y_train_oversampled)

y_pred = log_reg.predict(X_test)
#acc=log_reg.score(y_test,y_pred)
print(classification_report(y_test, y_pred))
accuracy=log_reg.score(X_test,y_test)
print(accuracy)
confusion_matrix(y_test,y_pred)

              precision    recall  f1-score   support

           0       0.97      0.74      0.84     58484
           1       0.22      0.77      0.34      5475

    accuracy                           0.74     63959
   macro avg       0.59      0.75      0.59     63959
weighted avg       0.91      0.74      0.80     63959

0.7414124673619037


array([[43212, 15272],
       [ 1267,  4208]])